# Fitting with Iminuit Vs. Emcee

This notebook provides a simple comparison of the `fit_lc` and `mcmc_lc` minimization routines available in `sncosmo` (based on `iminuit` and `emcee` respectively).

In [ ]:
import sncosmo
from matplotlib import pyplot as plt
from phot_class import fit_func_wraps, models, utils


For simplicity, we use the example data from sncosmo.

In [ ]:
data = sncosmo.load_example_data()

models.register_sources(force=True)
model = sncosmo.Model(sncosmo.get_source('sn91bg', version='hsiao_phase'))
model.set(z=data.meta['z'], t0=data.meta['t0'])

vparams = ['amplitude', 'x1', 'c']  # parameters of model to vary
bounds={'x1': (0.65, 1.25), 'c': (0, 1)}  # bounds on parameters


We run a fit using the `iminuit` minimization. To avoid some known bugs in `sncosmo` we use the wrapped version of `fit_lc` available in our analysis pipeline. 

In [ ]:
iminuit_result, iminuit_fit = fit_func_wraps.simple_fit(
    data, model, vparams, bounds=bounds)

xs, d = utils.calc_model_chisq(data, iminuit_result, iminuit_fit)
print(f'chisq / ndof = {xs} / {d} = {xs / d}')
print(f'ncall = {iminuit_result.ncall}')
print(f'message = {iminuit_result.message}', flush=True)

fig = sncosmo.plot_lc(data, model=iminuit_fit, errors=iminuit_result.errors)
plt.show()


Next, we fit the same data using an MCMC minimization. Note that we retrieve a much more reasonable value for the `c` parameter while maintaining a similar quality of fit (chi-squared).

In [ ]:
mcmc_result, mcmc_fit = fit_func_wraps.mcmc_fit(
    data, model, vparams, bounds=bounds)

xs, d = utils.calc_model_chisq(data, mcmc_result, mcmc_fit)
print(f'chisq / ndof = {xs} / {d} = {xs / d}')
print(f'samples = {len(mcmc_result.samples)}')

fig = sncosmo.plot_lc(data, model=mcmc_fit, errors=mcmc_result.errors)
plt.show()
